In [3]:
import numpy as np
import pandas as pd
# from bayes_opt import BayesianOptimization
import xgboost as xgb
from itertools import product
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn import model_selection,ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import time
from datetime import datetime
import random
# from sklearn_pandas import DataFrameMapper
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, Imputer, LabelBinarizer, MultiLabelBinarizer,LabelEncoder
from sklearn.cluster import KMeans
from scipy.stats.mstats import gmean
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
#input data
train_df=pd.read_json('../input/train.json').reset_index(drop = True)
test_df=pd.read_json('../input/test.json').reset_index(drop = True)

print train_df.shape
print test_df.shape

(49352, 15)
(74659, 14)


In [7]:
data_path = "../input/"
train_X_0322 = pd.read_csv(data_path + 'train_BM_MB_add03052240.csv')
test_X_0322 = pd.read_csv(data_path + 'test_BM_MB_add03052240.csv')

print train_X_0322.shape
print test_X_0322.shape

(49352, 322)
(74659, 322)


In [8]:
target_num_map = {'high':2, 'medium':1, 'low':0}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

In [11]:
data_path = "../input/"
train_X = pd.read_csv(data_path + 'train_CV_MS_52571.csv')
test_X = pd.read_csv(data_path + 'test_CV_MS_52571.csv')
print train_X.shape
print test_X.shape

(49352, 222)
(74659, 222)


In [12]:
time_feature = pd.read_csv(data_path + 'listing_image_time.csv')
time_feature.columns = ['listing_id','time_stamp']
train_X = train_X.merge(time_feature,on='listing_id',how='left')
test_X = test_X.merge(time_feature,on='listing_id',how='left')

print train_X.shape
print test_X.shape

(49352, 223)
(74659, 223)


In [18]:
def xgb_blend(estimators, train_x, train_y, test_x, fold, early_stopping_rounds=0):
    N_params = len(estimators)
    print ("Blend %d estimators for %d folds" % (N_params, fold))
    skf = KFold(n_splits=fold,random_state=1234)
    N_class = len(set(train_y))
        
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x_mean = np.zeros((test_x.shape[0], N_class*N_params))
    test_blend_x_gmean = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros ((fold,N_params))
    best_rounds = np.zeros ((fold, N_params))
    
    for j, est in enumerate(estimators):
        est.set_params(objective = 'binary:logistic')
        est.set_params(silent = False)
        est.set_params(learning_rate = 0.03)
        est.set_params(n_estimators=1000000)
        
        print ("Model %d: %s" %(j+1, est))

        test_blend_x_j = np.zeros((test_x.shape[0], N_class*fold))
    
        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x.iloc[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x.iloc[val_index]
            val_y_fold = train_y[val_index]      

            est.fit(train_x_fold,train_y_fold,
                    eval_set = [(val_x_fold, val_y_fold)],
                    eval_metric = 'logloss',
                    early_stopping_rounds=early_stopping_rounds,
                    verbose=False)
            best_round=est.best_iteration
            best_rounds[i,j]=best_round
            print ("best round %d" % (best_round))
            val_y_predict_fold = est.predict_proba(val_x_fold,ntree_limit=best_round)
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            
            test_blend_x_j[:,(i*N_class):(i+1)*N_class] = est.predict_proba(test_x,ntree_limit=best_round)
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))
            
        test_blend_x_mean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([test_blend_x_j[:,range(0,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(1,N_class*fold,N_class)].mean(1)]).T
        
        test_blend_x_gmean[:,(j*N_class):(j+1)*N_class] = \
                np.stack([gmean(test_blend_x_j[:,range(0,N_class*fold,N_class)], axis=1),
                          gmean(test_blend_x_j[:,range(1,N_class*fold,N_class)], axis=1)]).T
            
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x_mean, test_blend_x_gmean, scores,best_rounds)


# Low

In [19]:
y_low =[]
for i in range(train_X.shape[0]):
    y_low.append(1 if train_y[i] == 0 else 0)
    
y_low = np.array(y_low)  
print np.sum(y_low)

34284


In [20]:
estimators = [
            xgb.XGBClassifier(max_depth = 4,
                              min_child_weight = 6,
                              colsample_bytree = 0.290381 ,
                              subsample = 0.988734 ,
                              gamma = 0.075128)          
             ]
#  	 	max_depth 	min_child_weight 	colsample_bytree 	subsample 	gamma 	 	score
# 12 	4.697965 	6.439472 	 	 	0.290381 	 	 	0.988734 	0.075128 	-0.386062
# 5 	4.361961 	8.354384 	 	 	0.768667 	 	 	0.986623 	0.003024 	-0.386068
# 28 	5.020978 	19.866215 	 	 	0.883846 	 	 	0.984659 	0.135751 	-0.386150


(train_blend_x_xgb_low,
 test_blend_x_xgb_mean_low,
 test_blend_x_xgb_gmean_low,
 blend_scores_xgb_low,
 best_rounds_xgb_low) = xgb_blend(estimators,
                              train_X,y_low,
                              test_X,
                              5,
                              200)


Blend 1 estimators for 5 folds
Model 1: XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.290381,
       gamma=0.075128, learning_rate=0.03, max_delta_step=0, max_depth=4,
       min_child_weight=6, missing=None, n_estimators=1000000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=0.988734)
Model 1 fold 1
best round 2558
('Score: ', 0.38437436864385305)
Model 1 fold 1 fitting finished in 58.124s
Model 1 fold 2
best round 2164
('Score: ', 0.34849309257339706)
Model 1 fold 2 fitting finished in 49.444s
Model 1 fold 3
best round 2468
('Score: ', 0.34681978626416798)
Model 1 fold 3 fitting finished in 55.462s
Model 1 fold 4
best round 2581
('Score: ', 0.36662480416383131)
Model 1 fold 4 fitting finished in 56.222s
Model 1 fold 5
best round 1959
('Score: ', 0.38789336896971094)
Model 1 fold 5 fitting finished in 45.294s
Score for model 1 is 0.366841
Score for blended models is 0.366841

In [21]:
now = datetime.now()

name_train_blend = '../output/train_blend_xgb_low_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../output/test_blend_xgb_low_mean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../output/test_blend_xgb_low_gmean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_xgb_low,axis=0))
print (np.mean(best_rounds_xgb_low,axis=0))
np.savetxt(name_train_blend,train_blend_x_xgb_low, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_xgb_mean_low, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_xgb_gmean_low, delimiter=",")

[ 0.36684108]
[ 2346.]


# medium

In [22]:
y_medium =[]
for i in range(train_X.shape[0]):
    y_medium.append(1 if train_y[i] == 1 else 0)
    
y_medium = np.array(y_medium)  
print np.sum(y_medium)

11229


In [23]:
estimators = [
            xgb.XGBClassifier(max_depth = 5,
                              min_child_weight = 16,
                              colsample_bytree = 0.986761 ,
                              subsample = 0.986039 ,
                              gamma = 0.034931)          
             ]



(train_blend_x_xgb_medium,
 test_blend_x_xgb_mean_medium,
 test_blend_x_xgb_gmean_medium,
 blend_scores_xgb_medium,
 best_rounds_xgb_medium) = xgb_blend(estimators,
                              train_X,y_medium,
                              test_X,
                              5,
                              200)


Blend 1 estimators for 5 folds
Model 1: XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.986761,
       gamma=0.034931, learning_rate=0.03, max_delta_step=0, max_depth=5,
       min_child_weight=16, missing=None, n_estimators=1000000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=0.986039)
Model 1 fold 1
best round 992
('Score: ', 0.42220802852188838)
Model 1 fold 1 fitting finished in 62.282s
Model 1 fold 2
best round 848
('Score: ', 0.40444656485414893)
Model 1 fold 2 fitting finished in 55.550s
Model 1 fold 3
best round 1423
('Score: ', 0.40104374546981869)
Model 1 fold 3 fitting finished in 84.455s
Model 1 fold 4
best round 1080
('Score: ', 0.41120174265612414)
Model 1 fold 4 fitting finished in 66.330s
Model 1 fold 5
best round 1055
('Score: ', 0.42669308882738127)
Model 1 fold 5 fitting finished in 64.727s
Score for model 1 is 0.413119
Score for blended models is 0.413119


In [24]:
name_train_blend = '../output/train_blend_xgb_medium_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../output/test_blend_xgb_medium_mean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../output/test_blend_xgb_medium_gmean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_xgb_medium,axis=0))
print (np.mean(best_rounds_xgb_medium,axis=0))
np.savetxt(name_train_blend,train_blend_x_xgb_medium, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_xgb_mean_medium, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_xgb_gmean_medium, delimiter=",")

[ 0.41311863]
[ 1079.6]


# high

In [25]:
y_high =[]
for i in range(train_X.shape[0]):
    y_high.append(1 if train_y[i] == 2 else 0)
    
y_high = np.array(y_high)  
print np.sum(y_high)

3839


In [26]:
estimators = [
            xgb.XGBClassifier(max_depth = 6,
                              min_child_weight = 6,
                              colsample_bytree = 0.254931 ,
                              subsample = 0.998690 ,
                              gamma = 1.528020)          
             ]



(train_blend_x_xgb_high,
 test_blend_x_xgb_mean_high,
 test_blend_x_xgb_gmean_high,
 blend_scores_xgb_high,
 best_rounds_xgb_high) = xgb_blend(estimators,
                              train_X,y_high,
                              test_X,
                              5,
                              200)

Blend 1 estimators for 5 folds
Model 1: XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.254931,
       gamma=1.52802, learning_rate=0.03, max_delta_step=0, max_depth=6,
       min_child_weight=6, missing=None, n_estimators=1000000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=False, subsample=0.99869)
Model 1 fold 1
best round 837
('Score: ', 0.17048693307556323)
Model 1 fold 1 fitting finished in 30.681s
Model 1 fold 2
best round 775
('Score: ', 0.16771294040236134)
Model 1 fold 2 fitting finished in 29.559s
Model 1 fold 3
best round 634
('Score: ', 0.16826319966689912)
Model 1 fold 3 fitting finished in 25.273s
Model 1 fold 4
best round 674
('Score: ', 0.16382289716470466)
Model 1 fold 4 fitting finished in 26.635s
Model 1 fold 5
best round 933
('Score: ', 0.17342380741860161)
Model 1 fold 5 fitting finished in 33.723s
Score for model 1 is 0.168742
Score for blended models is 0.168742


In [27]:
name_train_blend = '../output/train_blend_xgb_high_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../output/test_blend_xgb_high_mean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_gmean = '../output/test_blend_xgb_high_gmean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'


print (np.mean(blend_scores_xgb_high,axis=0))
print (np.mean(best_rounds_xgb_high,axis=0))
np.savetxt(name_train_blend,train_blend_x_xgb_high, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_xgb_mean_high, delimiter=",")
np.savetxt(name_test_blend_gmean,test_blend_x_xgb_gmean_high, delimiter=",")

[ 0.16874196]
[ 770.6]


In [28]:
#combine

In [29]:
train_blend_x_xgb = np.vstack([train_blend_x_xgb_low[:,1],train_blend_x_xgb_medium[:,1],train_blend_x_xgb_high[:,1]]).T
train_blend_x_xgb.shape

(49352L, 3L)

In [30]:
test_blend_x_xgb_mean = np.vstack([test_blend_x_xgb_mean_low[:,1],test_blend_x_xgb_mean_medium[:,1],test_blend_x_xgb_mean_high[:,1]]).T
test_blend_x_xgb_mean.shape

(74659L, 3L)

In [31]:
test_blend_x_xgb_gmean = np.vstack([test_blend_x_xgb_gmean_low[:,1],test_blend_x_xgb_gmean_medium[:,1],test_blend_x_xgb_gmean_high[:,1]]).T
test_blend_x_xgb_gmean.shape

(74659L, 3L)

In [32]:
train_blend_x_xgb = pd.DataFrame(train_blend_x_xgb)
train_blend_x_xgb.columns = ["low", "medium", "high"]
train_blend_x_xgb["listing_id"] = train_X.listing_id.values

test_blend_x_xgb_mean = pd.DataFrame(test_blend_x_xgb_mean)
test_blend_x_xgb_mean.columns = ["low", "medium", "high"]
test_blend_x_xgb_mean["listing_id"] = test_X.listing_id.values

In [33]:
tmp_train = train_X_0322[['listing_id']].merge(train_blend_x_xgb,on = 'listing_id', how = 'left')[["low", "medium", "high"]].values
tmp_test_mean = test_X_0322[['listing_id']].merge(test_blend_x_xgb_mean,on = 'listing_id', how = 'left')[["low", "medium", "high"]].values

In [41]:
name_train_blend = '../output/train_blend_xgb_ovr_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../output/test_blend_xgb_ovr_mean_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



# print (np.mean(blend_scores_xgb,axis=0))
# print (np.mean(best_rounds_xgb,axis=0))
np.savetxt(name_train_blend,tmp_train, delimiter=",")
np.savetxt(name_test_blend_mean,tmp_test_mean, delimiter=",")

In [39]:
sub_name = '../output/sub_XGB_2bagging_CV_MS_52571_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(tmp_test_mean[:,:3])
out_df.columns = ["low", "medium", "high"]
out_df["listing_id"] = test_X_0322.listing_id.values
out_df.to_csv(sub_name, index=False)

In [40]:
tmp_test_mean[:10]

array([[  1.47930203e-01,   5.71099824e-01,   1.64883459e-01],
       [  9.77132773e-01,   1.00000978e-02,   8.71934639e-03],
       [  9.54789793e-01,   9.35790062e-02,   4.79983645e-03],
       [  1.54439420e-01,   5.56799817e-01,   2.12952900e-01],
       [  6.97897673e-01,   2.67950007e-01,   8.85903025e-03],
       [  7.56280649e-01,   2.96792227e-01,   5.74759413e-03],
       [  9.99139762e-01,   1.95185969e-03,   1.24549634e-04],
       [  2.58878088e-01,   4.86703187e-01,   1.73916987e-01],
       [  9.26443124e-01,   3.78795039e-02,   1.36201400e-02],
       [  9.85897434e-01,   1.59702195e-02,   1.36567574e-03]])

In [36]:
train_y

array([1, 0, 2, ..., 0, 0, 0], dtype=int64)